In [1]:
## tools
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper


In [2]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv,description="Query arxiv papers")
print(arxiv.name)

arxiv


In [3]:
arxiv.invoke("Attention is all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with low latencies remains challenging due to the quadratic input length\ncomplexity of the attention layers. In this work, we investigate the effect of\ndropping MLP and attention layers at inference time o"

In [4]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki,description="Query Wikipedia")
print(wiki.name)

wikipedia


In [5]:
from dotenv import load_dotenv
load_dotenv()

import os

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
# os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [6]:
## Tavily search tool
from langchain_community.tools.tavily_search import TavilySearchResults

tavily = TavilySearchResults()

In [7]:
tavily.invoke("What is the capital of France?")

[{'title': 'Paris facts: the capital of France in history',
  'url': 'https://home.adelphi.edu/~ca19535/page%204.html',
  'content': 'Paris is the capital of France, the largest country of Europe with 550 000 km2 (65 millions inhabitants). Paris has 2.234 million inhabitants end 2011.',
  'score': 0.8997663},
 {'title': 'Paris - Wikipedia',
  'url': 'https://en.wikipedia.org/wiki/Paris',
  'content': 'Paris (French pronunciation: [paʁi] ⓘ) is the capital and largest city of France. With an estimated population of 2,048,472 residents in January 2025[3] in an area of more than 105\xa0km2 (41\xa0sq\xa0mi),[4] Paris is the fourth-most populous city in the European Union, the ninth-most populous city in Europe and the 30th most densely populated city in the world in 2022.[5] Since the 17th century, Paris has been one of the world\'s major centres of finance, diplomacy, commerce, culture, fashion, and [...] As the capital of France, Paris is the seat of France\'s national government. For the

In [8]:
## combine all these tools in the list
tools = [arxiv, wiki, tavily]

In [9]:
## Initializse the LLM Model
from langchain_groq import ChatGroq

llm=ChatGroq(model="qwen-qwq-32b",api_key="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")


In [10]:
llm.invoke("What is the capital of India?")

AIMessage(content='\n<think>\nOkay, so I need to figure out what the capital of India is. Let me start by recalling any information I have about India\'s geography and politics. I know India is a big country in South Asia, and like many countries, it has a capital city where the government is based. But wait, I might be mixing up some details here. Let me think step by step.\n\nFirst, India\'s government is a federal parliamentary democratic republic, right? That usually means there\'s a capital city where the President, Prime Minister, and the Parliament are located. Now, what cities in India come to mind? Major cities like Mumbai, Delhi, Bangalore, Kolkata... Hmm, Mumbai was the capital before? Wait, I think there was a change. I remember hearing that the capital was shifted from Mumbai to somewhere else a long time ago. \n\nWait, Mumbai is in Maharashtra, but Delhi is a separate union territory. Delhi is a major city, but is it the capital? I think so. Let me verify. I recall that N

In [11]:
llm_with_tools=llm.bind_tools(tools=tools)


In [12]:
## Execute the LLM with tools
llm_with_tools.invoke("What is the recent news on ai?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rpb0', 'function': {'arguments': '{"query": "recent news on AI"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 151, 'prompt_tokens': 322, 'total_tokens': 473, 'completion_time': 0.366577141, 'prompt_time': 0.02244825, 'queue_time': 0.23591710700000001, 'total_time': 0.389025391}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_3796682456', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-217c3ade-1968-4327-ad95-67f671b84b1d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'recent news on AI'}, 'id': 'call_rpb0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 322, 'output_tokens': 151, 'total_tokens': 473})

In [13]:
llm_with_tools.invoke("What is machine learning?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2k01', 'function': {'arguments': '{"query": "machine learning"}', 'name': 'wikipedia'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 181, 'prompt_tokens': 319, 'total_tokens': 500, 'completion_time': 0.439637457, 'prompt_time': 0.036311947, 'queue_time': 0.49307971999999994, 'total_time': 0.475949404}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_512a3da6bb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ce03be26-842c-496a-a514-0553284ed488-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'machine learning'}, 'id': 'call_2k01', 'type': 'tool_call'}], usage_metadata={'input_tokens': 319, 'output_tokens': 181, 'total_tokens': 500})

## Workflow

In [14]:
## State Schema
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage # human message or AI message
from typing import Annotated # labelling 
from langgraph.graph.message import add_messages # Reducers in Langgraph


In [15]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages] # list of messages
    

In [16]:
## Entire Chatbot with Langgraph
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition


In [29]:
## Node Defenition
def tool_calling_llm(state:State):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

# Build graph
builder=StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools",ToolNode(tools))


In [ ]:
## Edges
builder.add_edge(START,"tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message(result) from assistant is a tool call --> tools_conditon routes to tools
    # If the latest message(result) from assistant is not a tool call --> END
    tools_condition,
)
builder.add_edge("tools",END)

graph=builder.compile() 
display(Image(graph.get_graph().draw_mermaid_png()))

Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.


ValueError: Branch with name `tools_condition` already exists for node `tool_calling_llm`

{'tools_condition': Branch(path=tools_condition(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), ends={'tools': 'tools', '__end__': '__end__'}, then=None, input_schema=None)}
